![img](https://www.sc-best-practices.org/_images/adata.jpg)

In [1]:
import numpy as np
import pandas as pd
import anndata as ad 
from scipy.sparse import csr_matrix
# import plotly.express as px
# import matplotlib.pyplot as plt
import scanpy as sc
from IPython.display import display

# Full data exploration, no heavy processing

In [ ]:
adata = ad.io.read_h5ad('../data/GSE194122_openproblems_neurips2021_cite_BMMC_processed.h5ad')

AttributeError: module 'anndata' has no attribute 'io'

In [ ]:
adata.

## X

In [ ]:
adata.X

## Var

In [ ]:
adata.var

\#question why gene_id is sometimes Nan?  
\#question what do index names mean?

In [ ]:
if not adata.var.index.is_unique:
    display(adata.var[adata.var.index.duplicated(keep=False)].sort_index())

In [ ]:
adata.var['feature_types'].value_counts()

In [ ]:
adata.var['gene_id'].value_counts()

In [ ]:
adata.var.query('gene_id.isna()')['feature_types'].unique()

## obs

In [ ]:
adata.obs.head()

In [ ]:
adata.obs.columns

In [ ]:
adata.obs['cell_type'].value_counts()

In [ ]:
adata.obs['cell_type'].nunique()

In [ ]:
adata.obs['batch'].value_counts()

In [ ]:
adata.obs['Modality'].value_counts()

In [ ]:
adata.obs['Samplename'].value_counts()

## uns

In [ ]:
adata.uns

## obsm

In [ ]:
adata.obsm['ADT_X_pca'].shape

In [ ]:
adata.obsm['ADT_X_umap'].shape

In [ ]:
adata.obsm['ADT_isotype_controls']

In [ ]:
adata.obsm['GEX_X_pca'].shape

In [ ]:
adata.obsm['GEX_X_umap'].shape

## layers

In [ ]:
adata.layers['counts']

\#question: what are layers?

# Multigrate part

## Subsample

In [ ]:
sc.pp.subsample(adata, n_obs=20000)
adata

In [ ]:
rna = adata[:, adata.var["feature_types"] == "GEX"].copy()
adt = adata[:, adata.var["feature_types"] == "ADT"].copy()

## RNA preprocessing

In [ ]:
rna.X, rna.layers["counts"]

\#question once again, what is the difference between layers and X ?

In [ ]:
rna.X = rna.layers["counts"].copy()
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)

n_top_genes = 2000
batch_key = "Site"

sc.pp.highly_variable_genes(rna, n_top_genes=n_top_genes, batch_key=batch_key, subset=True)
rna

## ADT processing

In [ ]:
import muon
import multigrate as mtg

In [ ]:
adt.X = adt.layers["counts"].copy()
muon.prot.pp.clr(adt)
adt.layers["clr"] = adt.X.copy()

In [ ]:
adata = mtg.data.organize_multimodal_adatas(
    adatas=[[rna], [adt]],  # a list of adata objects per modality, RNA-seq always goes first
    layers=[["counts"], ["clr"]],  # if need to use data from .layers, if None use .X
)
adata

In [ ]:
np.random.seed(42)
batches = np.random.choice(adata.obs['batch'].unique(), 3, replace=False)
genes = adata.var['gene_id'].sample(1000)
adata = adata[adata.obs['batch'].isin(batches), adata.var['gene_id'].isin(genes)]

adata.write_h5ad('../data/cite_filter.h5ad')

adata = ad.io.read_h5ad('../data/cite_filter.h5ad')

In [ ]:
adata.var_names_make_unique()

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20)

In [ ]:
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")

In [ ]:
sc.pl.pca(adata, color=['batch', 'cell_type'])

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

In [ ]:
sc.tl.umap(adata)
sc.pl.umap(adata, color=['batch', 'cell_type'])